**This notebook focuses on getting Spotify song data based on Kaggle dataset (same source as the larger one for artists used in notebook 4_) as well as using spotipy to pull data on any unmatched songs. Spoiler alert: I used Kaggle dataset that left out the "spotify:track:" on the ids; thus I only took the last 15 characters. This caused issues later when trying to get the track features. Notebook 6_...v2 is a redo to account for this issue**

#### Next Steps
- Continue down artist path - join with kaggle dataset
    - https://www.kaggle.com/yamaerenay/spotify-dataset-19212020-160k-tracks?select=data_by_artist_o.csv
    - data_by_artist_o.csv
    - but this is by artist name... 
- Continue down artist path - use Spotify API to gather this same info, but using API and IDs collected
    - would make the most sense
- Go down genre path
    - https://www.kaggle.com/yamaerenay/spotify-dataset-19212020-160k-tracks?select=data_by_artist_o.csv
    - data_by_genres_o.csv
- **Try same exercise with artists but for songs - can't zip both, but can search songs and take top title**
    - need to look more into what info to use
    - tracks.csv by track id and track name, artist id and artist name - has popularity/dancibility/energy
    - data_o.csv by track id, has artist name, has more audio information

#### Next step was trying the same exercise but for songs.

In [2]:
# Basics 
import pandas as pd
import numpy as np

# Text Cleaning
import re

#Spotify
# Source: https://betterprogramming.pub/how-to-extract-any-artists-data-using-spotify-s-api-python-and-spotipy-4c079401bc37
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time 

# Importing dfs with heavy processing
import pickle

In [3]:
# import peloton data
# Thank you to okaykristinakay on Reddit for the data!
# Source: https://www.reddit.com/r/pelotoncycle/comments/m18xnr/peloton_class_list_march_update/
df = pd.read_excel('../../../data/original_datasets/AGF_Peloton Classes March.xlsx')
df.head()

,classId,className,classDescription,classDifficulty,classDuration,classType,classLength,classLocation,classOriginalAirdate,classRating,classRatingCount,instructorName,instructorBio,classEquipment,classSongs,classArtists,classUrl
0,7f66378211c9476b9b5619bf989f91d0,20 min Peace Meditation,A guided meditation that focuses on cultivatin...,4.3333,20,Meditation,23,psny-studio-2,2021-09-03 13:25:00,0.9847,131,Aditi Shah,"To Aditi, yoga goes beyond movement and can br...",Yoga Block,Meditation 22,RIOPY,https://members.onepeloton.com/classes/bootcam...
1,54ac61803b364b2fa8378acd9f593cdb,15 min Bodyweight Strength,"No equipment, no problem. Join us for a high-e...",5.7755,15,Strength,19,psny-studio-3,2021-09-03 13:19:00,0.9933,297,Olivia Amato,"Born and raised in New York, Olivia grew up pl...",Workout Mat,"California Gurls,Let's Get Loud,Let It Rock (f...","Katy Perry,Snoop Dogg,Jennifer Lopez,Kevin Rud...",https://members.onepeloton.com/classes/bootcam...
2,c75fd4831573483c9d45739aae11d083,20 min Focus Flow: Lower Body,This yoga flow class focuses on poses that eng...,4.3664,20,Yoga,23,psny-studio-2,2021-09-03 12:51:00,1.0000,159,Aditi Shah,"To Aditi, yoga goes beyond movement and can br...","Yoga Blanket,Yoga Block,Yoga Mat","Interlude No 1,Oceansize,She Just Likes To Fig...","James Vincent McMorrow,Oh Wonder,Four Tet,Grim...",https://members.onepeloton.com/classes/bootcam...
3,470086936f7a4723ab5a53cb80b571ff,45 min Pop Bootcamp,Split your workout 50/50 between cardio on the...,7.8312,45,Tread Bootcamp,50,psny-studio-4,2021-09-03 11:56:00,0.9737,152,Olivia Amato,"Born and raised in New York, Olivia grew up pl...","Workout Mat,Medium Weights",34+35 (Remix) (feat. Doja Cat & Megan Thee Sta...,"Ariana Grande,Doja Cat,Megan Thee Stallion,Jus...",https://members.onepeloton.com/classes/bootcam...
4,9680a817bf2149d2b91990c87166a400,20 min Pop Ride,We dare you not to dance as you ride to all th...,7.4000,20,Cycling,24,uk,2021-09-03 07:52:00,1.0000,82,Sam Yo,Sam is a pro at many things but shines when it...,None,"Señorita,Marry You,Irreplaceable,What Do You M...","Justin Timberlake,Bruno Mars,Beyoncé,Justin Bi...",https://members.onepeloton.com/classes/bootcam...


In [4]:
# Opening pickle files
# Use classId where you can as key for merging
base_clean_df = pd.read_pickle("../../../data/pickled_dfs/base_clean_df.pkl")
artist_df = pd.read_pickle("../../../data/pickled_dfs/artist_df.pkl")
artist_id_class_count = pd.read_pickle("../../../data/pickled_dfs/larger_spot_artist_id_class_count.pkl")
songs_df = pd.read_pickle("../../../data/pickled_dfs/songs_df.pkl")
song_class_count_df = pd.read_pickle("../../../data/pickled_dfs/song_class_count_df.pkl")
song_class_count_df.rename(columns={'Artist':'Song'}, inplace=True) # Quick clean rename column
class_type_reorg_df = pd.read_pickle("../../../data/pickled_dfs/class_type_reorg_df.pkl")
new_class_types_with_artists = pd.read_pickle("../../../data/pickled_dfs/new_class_types_with_artists.pkl")
# artist_id_class_count has 186 rows with 0LyfQWJT6nXafLPZqxe9Of - it means Various Artists
# and 40 errors

In [151]:
base_clean_df
# Difference from df:
# Dropped null 'instructorBio' rows - only 9
# Dropped rows with null classSongs/classArtists
# Dropped row value with classSongs as float (1 row)
# Populate rows with classType Cycling and classEquipment None to classEquipment = Bike

,classId,className,classDescription,classDifficulty,classDuration,classType,classLength,classLocation,classOriginalAirdate,classRating,classRatingCount,instructorName,instructorBio,classEquipment,classSongs,classArtists,classUrl
0,7f66378211c9476b9b5619bf989f91d0,20 min Peace Meditation,A guided meditation that focuses on cultivatin...,4.3333,20,Meditation,23,psny-studio-2,2021-09-03 13:25:00,0.9847,131,Aditi Shah,"To Aditi, yoga goes beyond movement and can br...",Yoga Block,Meditation 22,RIOPY,https://members.onepeloton.com/classes/bootcam...
1,54ac61803b364b2fa8378acd9f593cdb,15 min Bodyweight Strength,"No equipment, no problem. Join us for a high-e...",5.7755,15,Strength,19,psny-studio-3,2021-09-03 13:19:00,0.9933,297,Olivia Amato,"Born and raised in New York, Olivia grew up pl...",Workout Mat,"California Gurls,Let's Get Loud,Let It Rock (f...","Katy Perry,Snoop Dogg,Jennifer Lopez,Kevin Rud...",https://members.onepeloton.com/classes/bootcam...
2,c75fd4831573483c9d45739aae11d083,20 min Focus Flow: Lower Body,This yoga flow class focuses on poses that eng...,4.3664,20,Yoga,23,psny-studio-2,2021-09-03 12:51:00,1.0000,159,Aditi Shah,"To Aditi, yoga goes beyond movement and can br...","Yoga Blanket,Yoga Block,Yoga Mat","Interlude No 1,Oceansize,She Just Likes To Fig...","James Vincent McMorrow,Oh Wonder,Four Tet,Grim...",https://members.onepeloton.com/classes/bootcam...
3,470086936f7a4723ab5a53cb80b571ff,45 min Pop Bootcamp,Split your workout 50/50 between cardio on the...,7.8312,45,Tread Bootcamp,50,psny-studio-4,2021-09-03 11:56:00,0.9737,152,Olivia Amato,"Born and raised in New York, Olivia grew up pl...","Workout Mat,Medium Weights",34+35 (Remix) (feat. Doja Cat & Megan Thee Sta...,"Ariana Grande,Doja Cat,Megan Thee Stallion,Jus...",https://members.onepeloton.com/classes/bootcam...
4,9680a817bf2149d2b91990c87166a400,20 min Pop Ride,We dare you not to dance as you ride to all th...,7.4000,20,Cycling,24,uk,2021-09-03 07:52:00,1.0000,82,Sam Yo,Sam is a pro at many things but shines when it...,Bike,"Señorita,Marry You,Irreplaceable,What Do You M...","Justin Timberlake,Bruno Mars,Beyoncé,Justin Bi...",https://members.onepeloton.com/classes/bootcam...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16743,a144fcf1595740caa610f6484deb3cdb,20 min Funk Fun Run,This 20 minute Fun Run is a musically driven c...,6.4567,20,Running,23,nyc-gw-tread,14/09/2018 10:22,0.9933,150,Becs Gentry,Becs joins Peloton from London as an accomplis...,None,"Man Funk (feat. Leron Thomas),If It Ain't Funk...","Guts,Leron Thomas,Soopasoul,FDEL,Basement Frea...",https://members.onepeloton.com/classes/bootcam...
16744,bd89a56c884b48e59ce9c43ab4f4d86f,45 min 90s Pop Ride,The 90s were a booming and inventive time in m...,8.1481,45,Cycling,49,nyc,13/09/2018 23:19,0.9925,1877,Denis Morton,"Raised in Florida, schooled in Tennessee, stee...",Light Weights,"Too Close - Radio Edit,Motownphilly - Original...","Next,Boyz II Men,Blues Traveler,Eagle-Eye Cher...",https://members.onepeloton.com/classes/bootcam...
16745,e5a99c83d7fc4f0686c68b02e5932204,10 min Arms Toning,Join Jess as she takes you through this 10-min...,7.4028,10,Strength,11,nyc,13/09/2018 21:02,0.9978,4481,Jess King,Jess is a charismatic instructor with a boundl...,Light Weights,"Hustle,Day and Night,Just Got Paid,Waiting for...","Daphne Willis,Gizzle,Lo Air,Sigala,Ella Eyre,M...",https://members.onepeloton.com/classes/bootcam...
16746,c57e123004d04df1bfae94c5e4399bb3,45 min 90s Ride,The 90s were a booming and inventive time in m...,8.3982,45,Cycling,49,nyc,13/09/2018 11:53,0.9922,2307,Emma Lovewell,Emma Lovewell is a Martha’s Vineyard native an...,Light Weights,"Show Me Love - Radio Version,Don't Speak,Gonna...","Robyn,No Doubt,C & C Music Factory,Technotroni...",https://members.onepeloton.com/classes/bootcam...


Note:
classId = 8840f82cdc624aa28a5d1babc51a3916 is the one you need to watch out for (has float classSong)

In [152]:
# Check lengths of dfs
print(f'artist_df: {len(artist_df)}')
print(f'artist_id_class_count: {len(artist_id_class_count)}')
print(f'songs_df: {len(songs_df)}')
print(f'song_class_count_df: {len(song_class_count_df)}')
print(f'class_type_reorg_df: {len(class_type_reorg_df)}')
print(f'new_class_types_with_artists: {len(new_class_types_with_artists)}')
# No inconsistencies where there shouldn't be

artist_df: 16748
artist_id_class_count: 10941
songs_df: 16748
song_class_count_df: 29904
class_type_reorg_df: 16748
new_class_types_with_artists: 16748


#### Notes:
- artist_df has a row for each of the 16748 classes with the lists of artists in that class and 10943 columns
    - column for classId, classArtists (from base_clean_df)
    - then each of the 10941 artists

- artist_id_class_count has 10941 rows for each of the artists with their class count, followers, genres, popularity
    - keep in mind nulls not cleaned yet...
    - keep them in there until you decide what you're doing with the data
    - can clean after merging with final data set


#### Before cleaning Song List try to match to Kaggle Spotify df for track IDs

In [153]:
# Import Kaggle Spotify track IDs for matching
# Source: https://www.kaggle.com/yamaerenay/spotify-dataset-19212020-160k-tracks?select=data_by_artist_o.csv
kag2_spot_track_df = pd.read_csv('tracks.zip', compression='zip')
kag2_spot_track_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4


In [154]:
# Rename Song column 'name' to prep for merge
song_class_count_df.rename(columns={'Song':'name'}, inplace=True)

In [155]:
song_class_count_spotid = song_class_count_df.merge(kag2_spot_track_df, on='name', how='left', indicator=True)

In [156]:
song_class_count_spotid['_merge'].value_counts()
# 60157 total rows now; duplicate rows added

both          41048
left_only     19109
right_only        0
Name: _merge, dtype: int64

In [157]:
# View duplicates
song_class_count_spotid[song_class_count_spotid['name'].duplicated(keep='first')].sort_values(by='name')['name']

26802                        #1
26803                        #1
25491                        #3
39944                       #41
30817           'Round Midnight
                  ...          
36414                       pov
8114     through the late night
10984           Échame La Culpa
38900           Über den Wolken
38901           Über den Wolken
Name: name, Length: 30253, dtype: object

In [158]:
# Drop duplicate rows
# Peloton classes use songs viewers know
# Take first spotify id matched with for 30253 rows
# Can also test this data set completey reliant on Spotify API searches...
# Would take forever...
song_class_count_spotid.drop_duplicates(subset='name', keep='first', inplace=True)

In [159]:
# Reset index post drops
song_class_count_spotid = song_class_count_spotid.reset_index(drop=True)

In [160]:
# Check length - back to original count, duplicates removed
song_class_count_spotid

,name,Class Count,id,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,_merge
0,,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,Weekend,2,67UJGroXxC22oxzqO8bozE,25.0,113107.0,0.0,['Eddie Cochran'],['1p0t3JtUTayV2wb1RGN9mO'],1962-01-01,0.738,...,-8.349,1.0,0.0312,0.4760,0.002770,0.294,0.959,98.779,4.0,both
2,Supalonely (feat. Gus Dapperton),1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,Play (Artful Dodger Mix - Main Mix Radio),1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,Light Your Ass On Fire (Club Mix) (feat. Pharr...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29899,Wish You Were Here (feat. Brandyn Burnette) [E...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
29900,Modern Jesus (Charli XCX Remix),2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
29901,Sandstorm - Komytea Radio Remix,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
29902,Just To Keep You Satisfied (Album Version),1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [161]:
song_class_count_spotid['_merge'].value_counts()
# 36% matched
# Interested, would have thought more would match
# But still saves processing power

left_only     19109
both          10795
right_only        0
Name: _merge, dtype: int64

In [162]:
leftover_songs = list(song_class_count_spotid[song_class_count_spotid['id'].isnull()]['name'])

In [163]:
leftover_songs

['',
 'Supalonely (feat. Gus Dapperton)',
 'Play (Artful Dodger Mix - Main Mix Radio)',
 'Light Your Ass On Fire (Club Mix) (feat. Pharrell)',
 'Moon and Star',
 "Let's Stay Together - 2015 Remastered Version",
 'There for You feat. Effie (Crush Club Remix)',
 'I Feel for You (Axwell Remix)',
 'I Like It - Dillon Francis Remix',
 'Din Daa Daa',
 'Beardo (Radio Edit)',
 'Ready For Your Love (CLOSE Ready For Your Dub) (feat. MNEK)',
 "When It's Over (Remastered)",
 'Mullholland',
 'Lose Sight',
 'O-H-I-O',
 'Piano Sonata No. 14 in C sharp minor',
 'Get Off Of My Cloud ((Original Single Mono Version))',
 'Turn The Light',
 'Another Brick in the Wall',
 ' Baby (Single Version)',
 'CoCo (MAKJ Remix)',
 'Get It Off (That Kid Chris Edit)',
 'Good Together - Alex Metric Remix',
 'Big Gucci',
 'Back From Hell (Remix)',
 'Summertime Sadness [Lana Del Rey vs. Cedric Gervais] - Cedric Gervais Remix',
 'I Will Survive (Album Version)',
 "Don't Let Go (Love) [Radio Edit]",
 'I Will Always Love You (

Clean leftover songs for Spotify API work
- DO NOT REMOVE PUNCTUATION

In [164]:
# Remove leading and trailing spaces from values
leftover_songs = [x.strip() for x in leftover_songs]

In [165]:
leftover_songs = sorted(leftover_songs)

In [166]:
# First value is a blank space
leftover_songs = leftover_songs[1:]

In [167]:
len(leftover_songs)

19108

### Spotify API Work

In [244]:
#Spotify API set up
# Source: https://betterprogramming.pub/how-to-extract-any-artists-data-using-spotify-s-api-python-and-spotipy-4c079401bc37
client_id = '4aac5b215c8a4fc591b506b71af7ebf5'
client_secret = '31d999b609424f569382b047ba828b00'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [169]:
# # Create function to pull out track features
# # Source: https://betterprogramming.pub/how-to-extract-any-artists-data-using-spotify-s-api-python-and-spotipy-4c079401bc37
# # Edited to put track information as additional row in track_df
# def getTrackFeatures(id):
#     meta = sp.track(id)
#     features = sp.audio_features(id)
    
#     # meta
#     id_searched = id
#     name = meta['name']
#     album = meta['album']['name']
#     artist = meta['album']['artists'][0]['name']
#     release_date = meta['album']['release_date']
#     length = meta['duration_ms']
#     popularity = meta['popularity']
    
#     # features
#     acousticness = features[0]['acousticness']
#     danceability = features[0]['danceability']
#     energy = features[0]['energy']
#     instrumentalness = features[0]['instrumentalness']
#     liveness = features[0]['liveness']
#     loudness = features[0]['loudness']
#     speechiness = features[0]['speechiness']
#     tempo = features[0]['tempo']
#     time_signature = features[0]['time_signature']
    
#     track = [id_searched, name, album, artist, release_date, length, popularity, danceability, acousticness, 
#              danceability, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature]
    
#     track_df.loc[(len(track_df) +1)] = track
#     return track_df

In [170]:
# Set up search term 
searchtest = leftover_songs[0]
searchtest

'"(Mucho Mambo) Sway"'

In [171]:
# Pull out songid
result = sp.search(searchtest)
songidtest = result['tracks']['items'][0]['uri']
# artistidtest = result['tracks']['items'][0]['album']['artists'][0]['uri']
songidtest[15:]

'99en6xJ7unV8IluPBAYch'

In [172]:
# Create new df for songs and id capture
leftover_songs_df = pd.DataFrame(leftover_songs, columns=['song'])

# Create new column for Spotify Ids pulled through function
leftover_songs_df['spotifyId_man'] = ""

In [173]:
# Create function to get song id
def getSongId(song):
    searchterm = song
    try:
        result = sp.search(searchterm)
        songid = result['tracks']['items'][0]['uri']
    except: # if do not find match, error
        return "error"
        pass
    else: # if do find match, return id
        return songid[15:]

In [174]:
# for num in range(2,11):
#     print(num)

In [175]:
# leftover_songs_df['spotifyId_man'][:1001]
# #1001 - is through row 1000
# #1001:2001 is 1001-2000

In [176]:
# leftover_songs_df['spotifyId_man'][:1001] = leftover_songs_df['song'][:1001].map(lambda a: getSongId(a))
# first_1k = leftover_songs_df[:1001]

In [177]:
# first_1k

In [178]:
# for num in range(2,11):
#     search_start = ((num-1)*1000)+1
#     search_end = (num*1000)+1
#     leftover_songs_df['spotifyId_man'][search_start:search_end] = leftover_songs_df['song'][search_start:search_end].map(lambda a: getSongId(a))
#     print(f'{num} [{search_start}:{search_end}]')

In [179]:
# leftover_songs_df['spotifyId_man'][1001:2001] = leftover_songs_df['song'][1001:2001].map(lambda a: getSongId(a))
# songs_1k_to_2k = leftover_songs_df[1001:2001]

In [180]:
# leftover_songs_df['spotifyId_man'][2001:3001] = leftover_songs_df['song'][2001:3001].map(lambda a: getSongId(a))
# songs_2k_to_3k = leftover_songs_df[2001:3001]

In [181]:
# leftover_songs_df['spotifyId_man'][3001:4001] = leftover_songs_df['song'][3001:4001].map(lambda a: getSongId(a))
# songs_3k_to_4k = leftover_songs_df[3001:4001]

In [182]:
# leftover_songs_df['spotifyId_man'][4001:5001] = leftover_songs_df['song'][4001:5001].map(lambda a: getSongId(a))
# songs_4k_to_5k = leftover_songs_df[4001:5001]

In [183]:
# frames = [first_1k, songs_1k_to_2k, songs_2k_to_3k, songs_3k_to_4k, songs_4k_to_5k]
# first_5k_songs = pd.concat(frames)

In [184]:
# first_5k_songs

In [185]:
# leftover_songs_df['spotifyId_man'][5001:6001] = leftover_songs_df['song'][5001:6001].map(lambda a: getSongId(a))
# songs_5k_to_6k = leftover_songs_df[5001:6001]

In [186]:
# leftover_songs_df['spotifyId_man'][6001:7001] = leftover_songs_df['song'][6001:7001].map(lambda a: getSongId(a))
# songs_6k_to_7k = leftover_songs_df[6001:7001]

In [187]:
# leftover_songs_df['spotifyId_man'][7001:8001] = leftover_songs_df['song'][7001:8001].map(lambda a: getSongId(a))
# songs_7k_to_8k = leftover_songs_df[7001:8001]

In [188]:
# leftover_songs_df['spotifyId_man'][8001:9001] = leftover_songs_df['song'][8001:9001].map(lambda a: getSongId(a))
# songs_8k_to_9k = leftover_songs_df[8001:9001]

In [189]:
# leftover_songs_df['spotifyId_man'][9001:10001] = leftover_songs_df['song'][9001:10001].map(lambda a: getSongId(a))
# songs_9k_to_10k = leftover_songs_df[9001:10001]

In [190]:
# frames = [songs_5k_to_6k, songs_6k_to_7k, songs_7k_to_8k, songs_8k_to_9k, songs_9k_to_10k]
# second_5k_songs = pd.concat(frames)

In [191]:
# second_5k_songs

In [192]:
# frames = [first_5k_songs, second_5k_songs]
# first_10k_songs = pd.concat(frames)

In [193]:
# # pickle df - done
# first_10k_songs.to_pickle("./first_10k_songs.pkl")

In [194]:
# leftover_songs_df['spotifyId_man'][10001:11001] = leftover_songs_df['song'][10001:11001].map(lambda a: getSongId(a))
# songs_10k_to_11k = leftover_songs_df[10001:11001]

In [195]:
# leftover_songs_df['spotifyId_man'][11001:12001] = leftover_songs_df['song'][11001:12001].map(lambda a: getSongId(a))
# songs_11k_to_12k = leftover_songs_df[11001:12001]

In [196]:
# leftover_songs_df['spotifyId_man'][12001:13001] = leftover_songs_df['song'][12001:13001].map(lambda a: getSongId(a))
# songs_12k_to_13k = leftover_songs_df[12001:13001]

In [197]:
# leftover_songs_df['spotifyId_man'][13001:14001] = leftover_songs_df['song'][13001:14001].map(lambda a: getSongId(a))
# songs_13k_to_14k = leftover_songs_df[13001:14001]

In [198]:
# leftover_songs_df['spotifyId_man'][14001:15001] = leftover_songs_df['song'][14001:15001].map(lambda a: getSongId(a))
# songs_14k_to_15k = leftover_songs_df[14001:15001]

In [199]:
# frames = [songs_10k_to_11k, songs_11k_to_12k, songs_12k_to_13k, songs_13k_to_14k, songs_14k_to_15k]
# third_5k_songs = pd.concat(frames)

In [200]:
# third_5k_songs

In [201]:
# len(leftover_songs_df['spotifyId_man'])

In [202]:
# leftover_songs_df['spotifyId_man'][15001:16001] = leftover_songs_df['song'][15001:16001].map(lambda a: getSongId(a))
# songs_15k_to_16k = leftover_songs_df[15001:16001]

In [203]:
# leftover_songs_df['spotifyId_man'][16001:17001] = leftover_songs_df['song'][16001:17001].map(lambda a: getSongId(a))
# songs_16k_to_17k = leftover_songs_df[16001:17001]

In [204]:
# leftover_songs_df['spotifyId_man'][17001:18001] = leftover_songs_df['song'][17001:18001].map(lambda a: getSongId(a))
# songs_17k_to_18k = leftover_songs_df[17001:18001]

In [205]:
# leftover_songs_df['spotifyId_man'][18001:19001] = leftover_songs_df['song'][18001:19001].map(lambda a: getSongId(a))
# songs_18k_to_19k = leftover_songs_df[18001:19001]

In [206]:
# leftover_songs_df['spotifyId_man'][19001:19109] = leftover_songs_df['song'][19001:19109].map(lambda a: getSongId(a))
# songs_19k_to_end = leftover_songs_df[19001:19109]

In [207]:
# frames = [songs_15k_to_16k, songs_16k_to_17k, songs_17k_to_18k, songs_18k_to_19k, songs_19k_to_end]
# last_5k_songs = pd.concat(frames)

In [208]:
# frames = [third_5k_songs, last_5k_songs]
# last_10k_songs = pd.concat(frames)

In [209]:
# # pickle df - done
# last_10k_songs.to_pickle("./last_10k_songs.pkl")

In [210]:
# frames = [first_10k_songs, last_10k_songs]
# all_20k_songs = pd.concat(frames)
# all_20k_songs
# # really only 1908 rows

In [211]:
# # pickle df - done
# all_20k_songs.to_pickle("./all_20k_songs.pkl")

In [212]:
# leftover_songs_df = pd.read_pickle("./all_20k_songs.pkl")

In [213]:
# leftover_songs_df

In [214]:
# leftover_songs_df[leftover_songs_df['spotifyId_man']=='error']
# # 88.3% songs matched in leftovers

In [215]:
# song_class_count_spotid

In [216]:
# # Prep from merge rename column "song" with song title values as name
# leftover_songs_df.rename(columns={'song':'name'}, inplace=True)

In [217]:
# song_class_count_spotid.drop(columns='_merge', inplace=True)

In [218]:
# # repeat left join between original song_class_count_spotid and new leftover_songs_df based on song name
# song_class_count_spotid_2 = song_class_count_spotid.merge(leftover_songs_df, on='name', how='left', indicator=True)

In [219]:
# # Duplicates added, 9 dupes
# song_class_count_spotid_2['_merge'].value_counts()

In [220]:
# # View duplicate artists
# song_class_count_spotid_2[song_class_count_spotid_2['name'].duplicated(keep='first')].sort_values(by='name')['name']

In [221]:
# # Drop duplicate rows, take first spotify id matched with for 7 records
# song_class_count_spotid_2.drop_duplicates(subset='name', keep='first', inplace=True)

In [222]:
# # Reset index post drops
# song_class_count_spotid_2 = song_class_count_spotid_2.reset_index(drop=True)

In [223]:
# song_class_count_spotid_2

In [224]:
# # Replace nans in ['id'] with values found in manual Id pull (['spotifyId_man'])
# song_class_count_spotid_2['id'] = np.where(~song_class_count_spotid_2['id'].isnull(),
#                                            song_class_count_spotid_2['id'],song_class_count_spotid_2['spotifyId_man'])

In [225]:
# # Clean df by dropping 'spotifyId_man', '_merge' helper columns
# song_class_count_spotid_2.drop(columns=['spotifyId_man', '_merge'], inplace=True)

In [226]:
# song_class_count_spotid_2['id'].isnull().sum()

In [227]:
# 1-((song_class_count_spotid_2['id'].isnull().sum())/len(song_class_count_spotid_2))
# # 97% of songs have spotify Id - THIS STILL HAS ERRORS INCLUDED
# # 9

In [228]:
# len(song_class_count_spotid_2)

In [229]:
# song_class_count_spotid_2.describe().apply(lambda s: s.apply('{0:.5f}'.format))

In [230]:
# # pickle df - done
# song_class_count_spotid_2.to_pickle("./song_id_class_count.pkl")

In [231]:
# Bring in songs with track IDs table
song_id_class_count = pd.read_pickle("./song_id_class_count.pkl")

In [232]:
song_id_class_count[(song_id_class_count['id'] == 'error') | (song_id_class_count['id'].isnull())]

,name,Class Count,id,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Let's Stay Together - 2015 Remastered Version,1,error,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,Ready For Your Love (CLOSE Ready For Your Dub)...,1,error,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,Baby (Single Version),1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,I Will Always Love You (Hex Hector Radio Mix: ...,11,error,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29857,Now You See It (Benny Benassi Remix Radio Edit...,2,error,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29868,L. 75: III. Clair de lune,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29896,Heard 'Em Say (feat. Adam Levine),28,error,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29899,Wish You Were Here (feat. Brandyn Burnette) [E...,1,error,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now that most trackIds are solved for, get the song information

In [233]:
iderrortest = song_id_class_count.loc[12]['id']
idnulltest = song_id_class_count.loc[24]['id']

In [234]:
import math
math.isnan(idnulltest)

True

In [313]:
# Create df to hold Spotify track features returned from function below
track_df = pd.DataFrame(columns = ['id_searched', 'name', 'album', 'artist', 'release_date', 
                                   'length', 'popularity', 'danceability', 'acousticness', 
                                   'danceability', 'energy', 'instrumentalness', 'liveness', 
                                   'loudness', 'speechiness', 'tempo', 'time_signature'])

In [308]:
# Create function to pull out track features
# Source: https://betterprogramming.pub/how-to-extract-any-artists-data-using-spotify-s-api-python-and-spotipy-4c079401bc37
# Edited to put track information as additional row in track_df
def getTrackFeatures(id):
#     id = 'spotify:track:' + id
    try:
        meta = sp.track(id)
        features = sp.audio_features(id)

        # meta
        id_searched = id
        name = meta['name']
        album = meta['album']['name']
        artist = meta['album']['artists'][0]['name']
        release_date = meta['album']['release_date']
        length = meta['duration_ms']
        popularity = meta['popularity']

        # features
        acousticness = features[0]['acousticness']
        danceability = features[0]['danceability']
        energy = features[0]['energy']
        instrumentalness = features[0]['instrumentalness']
        liveness = features[0]['liveness']
        loudness = features[0]['loudness']
        speechiness = features[0]['speechiness']
        tempo = features[0]['tempo']
        time_signature = features[0]['time_signature']

        track = [id_searched, name, album, artist, release_date, length, popularity, danceability, acousticness, 
                 danceability, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature]
    except: # if do not find match, pass
        pass
    else: # if do find match, return id
        track_df.loc[(len(track_df) +1)] = track
        return track_df

In [237]:
# For reference for try/except
# def getSongId(song):
#     searchterm = song
#     try:
#         result = sp.search(searchterm)
#         songid = result['tracks']['items'][0]['uri']
#     except: # if do not find match, error
#         return "error"
#         pass
#     else: # if do find match, return id
#         return songid[15:]

In [238]:
song_ids = song_id_class_count[(song_id_class_count['id'] != 'error') & (~song_id_class_count['id'].isnull())]['id']

In [239]:
# Reset index
song_ids.reset_index(drop=True, inplace=True)

In [240]:
# Create DataFrame from Series
song_ids_df = song_ids.to_frame()

In [242]:
# Create list of Ids to test
first_100_songs = list(song_ids_df[:101]['id'])

In [272]:
# Many invalid IDs...
for song_id in first_100_songs:
    getTrackFeatures(song_id)

HTTP Error for GET to https://api.spotify.com/v1/tracks/GZoWLVbmxcBys6g0DLFLf returned 400 due to invalid id
HTTP Error for GET to https://api.spotify.com/v1/tracks/qA0jZKzOpfcpXoc1vDkC0 returned 400 due to invalid id
HTTP Error for GET to https://api.spotify.com/v1/tracks/YhfOgIyJJA2hBgTsHi9WK returned 400 due to invalid id
HTTP Error for GET to https://api.spotify.com/v1/tracks/xyanCylUYNcPJp1ao0PSb returned 400 due to invalid id
HTTP Error for GET to https://api.spotify.com/v1/tracks/RNJXELbt7D2tZafdNWS6C returned 400 due to invalid id
HTTP Error for GET to https://api.spotify.com/v1/tracks/9BZj3OG8zZHDlWe1dnDyy returned 400 due to invalid id
HTTP Error for GET to https://api.spotify.com/v1/tracks/WK7h2WfMmnX7zOZnfzoYo returned 400 due to invalid id
HTTP Error for GET to https://api.spotify.com/v1/tracks/IibbxjQER0Y1yKtS7wiQV returned 400 due to invalid id
HTTP Error for GET to https://api.spotify.com/v1/tracks/2vyPNKBFvbQUhnEYbwP5G returned 400 due to invalid id
HTTP Error for GET 

In [300]:
track_df
# 34 out of 100 were matched to a song
# This means my song id pulls did not get the ids needed...

0

## Revisited how TrackIDs are pulled

In [301]:
test_100_again = leftover_songs_df[:101]
test_100_again

,song,spotifyId_man
0,"""(Mucho Mambo) Sway""",
1,"""40 Day Dream""",
2,"""40""",
3,"""99""",
4,"""Che gelida manina"" from La Bohème",
...,...,...
96,1 Thing - Radio Version,
97,1 Thing - featuring Eve,
98,1-800-273-8255 (feat. Alessia Cara & Khalid),
99,1-800-Spice,


In [302]:
def getSongId2(song):
    searchterm = song
    try:
        result = sp.search(searchterm)
        songid = result['tracks']['items'][0]['uri']
    except: # if do not find match, error
        return "error"
        pass
    else: # if do find match, return id
        return songid

In [303]:
test_100_again['spotifyId_man'][:101] = test_100_again['song'][:101].map(lambda a: getSongId2(a))
test_100_songs = test_100_again[:101]

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/series.py:1240: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_with(key, value)
/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2878: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [304]:
test_100_songs

,song,spotifyId_man
0,"""(Mucho Mambo) Sway""",spotify:track:499en6xJ7unV8IluPBAYch
1,"""40 Day Dream""",spotify:track:4ek3pPdfvIbPxynmlisI0k
2,"""40""",spotify:track:0vtJ9Dq53hfXE3KyP1yPni
3,"""99""",spotify:track:7sLpSWxQazJzDVG6YGzlVs
4,"""Che gelida manina"" from La Bohème",spotify:track:6Px8dDwYhF0yogstmbr3sU
...,...,...
96,1 Thing - Radio Version,spotify:track:7iWikZuAbA8XVjDWSN8iZu
97,1 Thing - featuring Eve,spotify:track:0qlSHQbRffEDmp8QsEkBqZ
98,1-800-273-8255 (feat. Alessia Cara & Khalid),spotify:track:0Towf4igKUBdwmdFZ2S2GT
99,1-800-Spice,spotify:track:7Jv0JZb3ztniGWAd03J8gj


In [310]:
# Create list of Ids to test
test_100_songid_list = list(test_100_songs['spotifyId_man'])

In [315]:
# Many invalid IDs...
for song_id in test_100_songid_list:
    if (song_id == 'error'):
        continue
    else:
        getTrackFeatures(song_id)

In [316]:
track_df

,id_searched,name,album,artist,release_date,length,popularity,danceability,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
1,spotify:track:499en6xJ7unV8IluPBAYch,Mucho Mambo - Sway,Mucho Dance,Various Artists,2014-03-17,208360,48,0.625,0.403000,0.625,0.945,0.653000,0.4610,-4.240,0.0383,127.982,4
2,spotify:track:4ek3pPdfvIbPxynmlisI0k,40 Day Dream - 2019 - Remaster,Up From Below (10th Anniversary Edition) [2019...,Edward Sharpe & The Magnetic Zeros,2009-07-07,232690,60,0.560,0.102000,0.560,0.681,0.000000,0.0573,-4.875,0.0485,81.991,4
3,spotify:track:0vtJ9Dq53hfXE3KyP1yPni,40oz. To Freedom,40oz. To Freedom,Sublime,1992-06-01,183000,61,0.688,0.000530,0.688,0.489,0.002920,0.1430,-7.848,0.0378,94.270,4
4,spotify:track:7sLpSWxQazJzDVG6YGzlVs,99 Problems,The Black Album,JAY-Z,2003-11-14,234626,66,0.494,0.006610,0.494,0.887,0.000000,0.1030,-4.297,0.3980,89.554,4
5,spotify:track:6Px8dDwYhF0yogstmbr3sU,"Che Gelida Manina (From ""La Boheme - Act I"") -...",Live! Opera,Various Artists,2015-07-30,329252,3,0.198,0.980000,0.198,0.190,0.471000,0.6820,-15.264,0.0471,78.816,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,spotify:track:7iWikZuAbA8XVjDWSN8iZu,1 Thing (feat. Eve) - Radio Version,1 Thing EP,Amerie,2018-07-20,214973,44,0.591,0.105000,0.591,0.953,0.000000,0.0262,-4.027,0.3260,127.312,5
85,spotify:track:0qlSHQbRffEDmp8QsEkBqZ,Everyday With You Girl,The Things We Do for Love - 16 Love Songs of t...,Various Artists,2002-04-01,154293,1,0.695,0.216000,0.695,0.393,0.000000,0.0790,-12.594,0.0346,117.057,4
86,spotify:track:0Towf4igKUBdwmdFZ2S2GT,1-800-273-8255 (In the Style of Logic (feat. A...,36 Hip Hop Backing Tracks,Top Pop Instrumentals,2017-09-11,254400,2,0.611,0.205000,0.611,0.161,0.812000,0.1280,-18.816,0.0320,99.960,4
87,spotify:track:7Jv0JZb3ztniGWAd03J8gj,1-800-Spice,Spice 1,Spice 1,1992-04-14,243573,17,0.849,0.000277,0.849,0.448,0.000006,0.2250,-14.608,0.5050,163.840,4


In [318]:
test_500_again = leftover_songs_df[:501]

In [319]:
test_500_again['spotifyId_man'][:501] = test_500_again['song'][:501].map(lambda a: getSongId2(a))
test_500_songs = test_500_again[:501]

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/series.py:1240: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_with(key, value)
/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2878: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [320]:
# Create list of Ids to test
test_500_songid_list = list(test_500_songs['spotifyId_man'])

In [321]:
# Many invalid IDs...
for song_id in test_500_songid_list:
    if (song_id == 'error'):
        continue
    else:
        getTrackFeatures(song_id)

In [322]:
track_df

,id_searched,name,album,artist,release_date,length,popularity,danceability,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
1,spotify:track:499en6xJ7unV8IluPBAYch,Mucho Mambo - Sway,Mucho Dance,Various Artists,2014-03-17,208360,48,0.625,0.40300,0.625,0.945,0.653000,0.4610,-4.240,0.0383,127.982,4
2,spotify:track:4ek3pPdfvIbPxynmlisI0k,40 Day Dream - 2019 - Remaster,Up From Below (10th Anniversary Edition) [2019...,Edward Sharpe & The Magnetic Zeros,2009-07-07,232690,60,0.560,0.10200,0.560,0.681,0.000000,0.0573,-4.875,0.0485,81.991,4
3,spotify:track:0vtJ9Dq53hfXE3KyP1yPni,40oz. To Freedom,40oz. To Freedom,Sublime,1992-06-01,183000,61,0.688,0.00053,0.688,0.489,0.002920,0.1430,-7.848,0.0378,94.270,4
4,spotify:track:7sLpSWxQazJzDVG6YGzlVs,99 Problems,The Black Album,JAY-Z,2003-11-14,234626,66,0.494,0.00661,0.494,0.887,0.000000,0.1030,-4.297,0.3980,89.554,4
5,spotify:track:6Px8dDwYhF0yogstmbr3sU,"Che Gelida Manina (From ""La Boheme - Act I"") -...",Live! Opera,Various Artists,2015-07-30,329252,3,0.198,0.98000,0.198,0.190,0.471000,0.6820,-15.264,0.0471,78.816,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,spotify:track:29NFTA7YSLp1e9jAlRHI5i,Ain't Giving Up - Freejak Remix,Ain't Giving Up (Remixes) - EP,Craig David,2016-09-16,345560,18,0.679,0.00381,0.679,0.907,0.006390,0.0960,-5.366,0.0425,124.983,4
531,spotify:track:1iHttTPJV290P9SRTMYik2,Ain't Giving Up - Sigala Club Mix,Ain't Giving Up (Remixes) - EP,Craig David,2016-09-16,229813,31,0.690,0.07950,0.690,0.905,0.000197,0.0542,-1.230,0.0661,125.012,4
532,spotify:track:0cKe7EkDW29fM6RLyXdOHS,Ain't Giving Up - Extended Mix,Ain't Giving Up (Remixes) - EP,Craig David,2016-09-16,261120,17,0.773,0.00552,0.773,0.928,0.000727,0.2680,-3.588,0.0698,124.989,4
533,spotify:track:0I2SCrqGwtAic6COEdgRGB,Ain't Giving Up - WiDE AWAKE Remix,Ain't Giving Up (Remixes) - EP,Craig David,2016-09-16,209200,38,0.609,0.01970,0.609,0.646,0.000048,0.0787,-7.437,0.0513,74.987,4
